# Twitter Scraper / API

In [1]:
# Imports here
import twitter
import json
import tweepy







In [2]:
# Kuo











In [3]:
# Eddie



























In [23]:
# Yulong

consumer_key = 'xcW3f75Y6xEcoKnCrGDrNXSLG'
consumer_secret = 'SNQSDZeDlPoUSa443ElufrWxuCqdljfq6FME7Jin9DZdUQRTKa'
access_token = '153588498-8MoqCHZJG7jrLh8KToTErFAzpOzYMvZkhq1YzYmP'
access_token_secret = 'kw4SF72mY4bv5IuCDQxXQzdkfJMJ7v0t06F2OFMmwMfWT'

"""
api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token=access_token_key,
                  access_token_secret=access_token_secret)

results = api.GetSearch(
    raw_query='q=%23nra')

print type(results[0])
"""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)


def getTweets(hashtag, count):
    hashtag = '#' + hashtag
    tweets = []
    counter = 0
    for tweet in tweepy.Cursor(api.search, q=hashtag, count=count,
                               lang='en').items():
        tweets.append(tweet)
        if (counter > count):
            print 'Goes over the count'
            break;
        counter += 1
    return tweets

# tweets = getTweets('nra', 10)

# print len(tweets)

tweets[0]
print json.dumps(tweets[0]._json, sort_keys=True, indent=2)












{
  "contributors": null, 
  "coordinates": null, 
  "created_at": "Sat Mar 03 04:09:45 +0000 2018", 
  "entities": {
    "hashtags": [
      {
        "indices": [
          25, 
          37
        ], 
        "text": "2ADefenders"
      }, 
      {
        "indices": [
          39, 
          43
        ], 
        "text": "NRA"
      }
    ], 
    "media": [
      {
        "display_url": "pic.twitter.com/nF48JuxIHp", 
        "expanded_url": "https://twitter.com/DalinPaur/status/969753422630092800/photo/1", 
        "id": 969753419488653312, 
        "id_str": "969753419488653312", 
        "indices": [
          44, 
          67
        ], 
        "media_url": "http://pbs.twimg.com/media/DXVBmNhXcAAHdZ1.jpg", 
        "media_url_https": "https://pbs.twimg.com/media/DXVBmNhXcAAHdZ1.jpg", 
        "sizes": {
          "large": {
            "h": 507, 
            "resize": "fit", 
            "w": 750
          }, 
          "medium": {
            "h": 507, 
            "resiz

In [ ]:
# Xiaolun













